<a href="https://colab.research.google.com/github/kdwang1808/2020-MCM/blob/master/ABSA_Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 134448 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.17-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
import sys
os.chdir('drive/Colab Notebooks/2020MCM_PC')

In [0]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm as tqdm
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchsummary

In [64]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
hair = pd.read_csv('review_2/hair.csv')
micro = pd.read_csv('review_2/micro.csv')
baby = pd.read_csv('review_2/baby.csv')

In [13]:
hair.head(8)

,yy,mm,dd,product_parent,star_rating,helpful_votes,total_votes,review_headline,review_body
0,2008,1,1,201,5,7,8,Simply The Best Blow Dryer on The Market,This is the best standard blow dryer on the ma...
1,2008,1,7,165,5,4,6,My new best friend!,With a four year warranty I had to try it and ...
2,2008,1,8,75,5,16,17,"Terrific hair dryer, with caveat",Works great; same dryer used by my hair stylis...
3,2008,1,10,75,5,2,2,GREAT DRYER,This has cut down on my drying time and is ver...
4,2008,1,11,241,5,40,42,Conair 209R Ion Shine(s) for Under $20.00!,You can't get much for $20 these days. Howeve...
5,2008,1,12,241,1,11,14,Cord Keeper Breaks!,"I researched all the hairdryers on amazon, and..."
6,2008,1,13,180,4,5,6,Hard to believe it could be so good,I bought this dryer based on the user reviews ...
7,2008,1,28,244,4,0,0,conair hairdryer,It is okay. It doesn't get hot enough and the ...


In [0]:
hair['review'] = hair['review_headline']+hair['review_body']
hair = hair.drop(['review_headline', 'review_body'], axis=1)
micro['review'] = micro['review_headline']+micro['review_body']
micro = micro.drop(['review_headline', 'review_body'], axis=1)
baby['review'] = baby['review_headline']+baby['review_body']
baby = baby.drop(['review_headline', 'review_body'], axis=1)

In [17]:
baby.head()

,yy,mm,dd,product_parent,star_rating,helpful_votes,total_votes,review
0,2008,2,15,4383,5,1,2,Womb Sounds Bear is excellent for Newborn!I pu...
1,2008,3,16,1020,2,31,33,Beware if your baby sucks on everythingI loved...
2,2008,3,28,3846,3,0,6,had to get moreI had to get more of these afte...
3,2008,4,6,3349,5,0,0,WubbanuberifficWe have twin girls that are alm...
4,2008,6,5,4254,5,0,0,LOVE this pacifier!My daughter (3.5 mos old no...


In [0]:
hair.to_csv(r'hair.csv', index = False)
baby.to_csv(r'baby.csv', index = False)
micro.to_csv(r'micro.csv', index = False)

In [33]:
! python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [34]:
spacy.load('en_core_web_sm')

In [0]:
import spacy
from tqdm import tqdm
nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)

In [55]:
txt = 'I do not think you can beat this Sharp microwave'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [0]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [0]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [0]:
hair_clean=[]
for text in hair["review"]:
    hair_clean.append(clean_text(str(text)))
micro_clean=[]
for text in micro["review"]:
    micro_clean.append(clean_text(str(text)))
baby_clean=[]
for text in baby["review"]:
    baby_clean.append(clean_text(str(text)))

In [0]:
hair['clean'] = hair_clean
micro['clean'] = micro_clean
baby['clean'] = baby_clean

In [79]:
hair.head()

,yy,mm,dd,product_parent,star_rating,helpful_votes,total_votes,review,clean
0,2008,1,1,201,5,7,8,Simply The Best Blow Dryer on The MarketThis i...,simply best blow dryer marketthis best standar...
1,2008,1,7,165,5,4,6,My new best friend!With a four year warranty I...,new best friend four year warranty try love ta...
2,2008,1,8,75,5,16,17,"Terrific hair dryer, with caveatWorks great; s...",terrific hair dryer caveatworks great dryer us...
3,2008,1,10,75,5,2,2,GREAT DRYERThis has cut down on my drying time...,great dryerthis cut drying time gentle hair ex...
4,2008,1,11,241,5,40,42,Conair 209R Ion Shine(s) for Under $20.00!You ...,conair 209r ion shine 20 00 cannot get much 20...


In [81]:
baby_clean[10:20]

['happyi love new car seat third infant carrier owned far favorite things love easy adjust strap tightness fits infants 30 lbs seems comfortable infant easy clean fabric light color reflect light infants head flop sleeping comfortable handle easy put get car carseat base secures car well great price quality used old double graco stroller although fits tight things consider bigger carseats still fits camery backless booster seat side difficult buckle newborn since chest buckle big think heavy big carry around happy quality safe seat',
 'middle class sleeperthe porta crib ok bassinet much give pad give much support daughter would sleep ended puting porta crib mattress even though says seem give support finally slept like caddy diapers wipes although daughter three months already big changing table got paid definately good enough vacations hopefully big enough put bottom useful us',
 'greate productits great reasons 1 right left side could tuck vary important 2 washable 3 comfortable 4 wa

In [0]:
competitors = ['dry', 'dryer', 'baby', 'microwave', 'install']

In [84]:
aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in range(len(hair['review'])):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(hair['review'][x])) != 0:
        lines = str(hair['review'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')       
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)
            
            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])
            
    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
hair['compound_nouns'] = comp_terms
hair['aspect_keywords'] = aspect_terms
hair['competition'] = enemy
hair['competition_comp_nouns'] = ecomp_terms
hair.head()

,yy,mm,dd,product_parent,star_rating,helpful_votes,total_votes,review,clean,compound_nouns,aspect_keywords,competition,competition_comp_nouns
0,2008,1,1,201,5,7,8,Simply The Best Blow Dryer on The MarketThis i...,simply best blow dryer marketthis best standar...,[],[blown back],"[[dry, dryer]]","[(blow dryer, dryer)]"
1,2008,1,7,165,5,4,6,My new best friend!With a four year warranty I...,new best friend four year warranty try love ta...,"[(year warranty, warranty)]","[blowing apart, big mess]","[[dry], [dry, dryer], [dry], [dry, dryer], [dr...","[(size dryers, dryers), (hair dryers, dryers),..."
2,2008,1,8,75,5,16,17,"Terrific hair dryer, with caveatWorks great; s...",terrific hair dryer caveatworks great dryer us...,"[(year warranty, warranty), (year warranty, wa...","[Here 's, not honor, often used, sure looked, ...","[[dry, dryer], [dry, dryer], [dry, dryer], [dr...","[(hair dryer, dryer), (hair stylist, stylist),..."
3,2008,1,10,75,5,2,2,GREAT DRYERThis has cut down on my drying time...,great dryerthis cut drying time gentle hair ex...,[],[],[[dry]],[]
4,2008,1,11,241,5,40,42,Conair 209R Ion Shine(s) for Under $20.00!You ...,conair 209r ion shine 20 00 cannot get much 20...,"[(shot button, button), (air output, output), ...","[quickly removes, 's really, cold shot button,...","[[dry, dryer], [dry, dryer], [dry, dryer], [dr...","[(watt hair, hair), (hair dryer, dryer), (jet ..."


In [85]:
hair.tail(20)

,yy,mm,dd,product_parent,star_rating,helpful_votes,total_votes,review,clean,compound_nouns,aspect_keywords,competition,competition_comp_nouns
9681,2015,8,29,258,5,0,0,I am obsessedAfter using the same blowdryer fo...,obsessedafter using blowdryer probably 8 10yea...,"[(shoulder length, length), (length hair, hair)]",[],"[[dry, dryer], [dry], [dry, dryer], [dry, dryer]]","[(hair thickness, thickness)]"
9682,2015,8,29,289,5,0,0,Works great. Just wish it had a medium setting...,works great wish medium settingworks great wis...,[],"[medium settingWorks, medium setting, Just wish]",[],[]
9683,2015,8,29,72,5,0,0,No complaints!I blowdry my hair at least 3 tim...,complaints blowdry hair least 3 times week usi...,[],[],"[[dry], [dry, dryer]]",[]
9684,2015,8,29,218,5,0,0,"Five StarsLove this, keeps my curls nice and n...",five starslove keeps curls nice frizzes,[],[not frizzes],[],[]
9685,2015,8,29,243,2,0,0,Two StarsBonnet is too small,two starsbonnet small,[],[],[],[]
9686,2015,8,30,98,5,3,3,"Works great, I use it 5 days a weekReckon I ha...",works great use 5 days weekreckon year least 7...,[],"[nt overheat, nt cost, hot air]",[],[]
9687,2015,8,30,82,3,0,0,"The styling tip does not stay on,The styling t...",styling tip stay styling tip stay keeps fallin...,[],[],[[dry]],"[(styling tip, tip), (styling tip, tip)]"
9688,2015,8,30,133,4,0,0,"Blows like a jet engine!Nice hair dryer, dual ...",blows like jet engine nice hair dryer dual vol...,[],[],"[[dry, dryer]]","[(jet dryer, dryer), (engine!Nice dryer, dryer..."
9689,2015,8,30,229,2,1,1,Great drier...........mehGreat product... Took...,great drier mehgreat product took anywhere 30 ...,[],"[only blows, Great deal, cold air, later blows...",[[dry]],"[(blow drier, drier)]"
9690,2015,8,30,6,5,0,0,A good replacementLike this as well as the one...,good replacementlike well one,[],[had before],[],[]
